# Импорт основных библиотек из датасета

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import math
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

from sklearn import linear_model
from sklearn import metrics

# graph (по моему не нужен)
import networkx as nx
from pyvis.network import Network

PATHES = ["./with_closed_ations_as________select_app_2018_X5_STORES.csv", "./with_closed_ations_as________select_app_2019_X5_STORES.csv", "./with_closed_ations_as________select_app_2020_X5_STORES.csv"]
# df = pd.read_csv(PATH)
df = pd.DataFrame()
for path in PATHES:
    df1 = pd.read_csv(path)
    df = pd.concat([df, df1])
    
df['action_date'] = pd.to_datetime(df['action_date'])
df = df[df['position_name'] == 'Продавец-кассир']

# Изменение Bool значений

In [ ]:
df["is_active_action"] = df.is_active_action.replace({True: 1, False: 0})
df["is_open_action"] = df.is_open_action.replace({True: 1, False: 0})
df

# Статистика

In [ ]:
df = df[df['is_active_action'] == 1]

# Декораторы для времени
from functools import singledispatch
@singledispatch
def toDays(time_delta):
    return time_delta
@toDays.register
def _(time_delta: pd.Timedelta):
    return time_delta.total_seconds()/60/60/24
@toDays.register
def _(time_delta: pd.Series):
    return time_delta.dt.total_seconds()/60/60/24


# Сортируем по аппликантам и по времени
df.sort_values(by=['id_applicant','action_date'], ascending = [False,True], inplace = True)
df.reset_index(drop = True, inplace = True)

# action_time
indexes = pd.RangeIndex(len(df) - 1)
df['action_time'] = toDays(df.iloc[indexes + 1]['action_date'].reset_index(drop = True) - df['action_date']) # In days
df['action_time'].iloc[df[df['id_applicant'] !=  df.iloc[indexes+1].append({'id_applicant': 0}, ignore_index = True)['id_applicant'].reset_index(drop = True)].index] = None

# Cреднее по принят
df['is_fits'] = 0
df.loc[df['result_name'] == 'Принят', 'is_fits'] = 1
print('Среднее по принят ')
df['is_fits'].mean()

In [ ]:
applicants_fits =  df.groupby(['id_applicant'])['is_fits'].sum()
applicants_fits.describe()

In [ ]:
df['id_applicant'][df['id_applicant'].isin(applicants_fits[applicants_fits > 2].index)].unique()
# Очень странные значения!!! Когда два раза принимают там в основном второе оформление - после выхода на работу
# Можете смотреть тут
# df[df['id_applicant'].isin([4041842])]

In [ ]:
df_fits = df[df['id_applicant'].isin(applicants_fits[applicants_fits > 0].index)].reset_index()
df_fits = df_fits[df_fits['is_active_action'] == 1]
df_fits.describe()

In [ ]:
len(df_fits['id_applicant'].unique())

In [ ]:
df_fits['Date_int'] = df_fits['action_date'].values.astype('int64')
partner_applicant =  df_fits.groupby(['id_partner','id_applicant'])['Date_int'].max().reset_index()
amount_applicant =  partner_applicant.groupby(['id_partner'])['Date_int'].max().reset_index()
partner_applicant = partner_applicant[partner_applicant['Date_int'].isin(amount_applicant['Date_int'])]
df_fits = pd.DataFrame(df_fits).drop(columns = 'Date_int')

df_fits = df_fits[df_fits['id_applicant'].isin(partner_applicant['id_applicant'])].reset_index(drop = True)

len(df_fits['id_applicant'].unique())

In [ ]:
# applicant_min =  df_fits.groupby(['id_applicant'])['action_date'].min().reset_index()
fits_max =  df_fits[df_fits['result_name'] == 'Принят'].groupby(['id_applicant'])['action_date'].max().reset_index()
fits_max

# unused for now
# new_tth = pd.DataFrame(applicant_min).drop(columns = 'action_date')
# new_tth['tth'] = toDays(fits_min['action_date'] - applicant_min['action_date'])

In [ ]:
df_fits.loc[df_fits['action_date'].isin(fits_max['action_date']) & (df_fits['action_time'].notna()), 'after_fit'] = 1
while True:
    after_indexes = df_fits[(df_fits['action_time'].notna())& (df_fits['after_fit'] == 1)].index
    after_fit_mask = (df_fits.index.isin(after_indexes + 1) & (df_fits['action_time'].notna())) & df_fits['after_fit'].isna()
    print(len(df_fits.loc[after_fit_mask]))
    if len(df_fits.loc[after_fit_mask]) == 0:
        break
    df_fits.loc[after_fit_mask, 'after_fit'] = 1

# Закомменченые полезные штуки из статистики

In [ ]:
# partner_applicant =  df_fits.groupby(['id_partner','id_applicant']).size().reset_index()
# amount_applicant =  partner_applicant.groupby(['id_partner'])['id_applicant'].count().reset_index()
# partner_applicant.set_index('id_partner', inplace = True)
# amount_applicant[amount_applicant['id_applicant']>2]

# amount_applicant['id_partner'][amount_applicant['id_applicant']>2].array


# df_fits.loc[df_fits.index.isin(after_indexes + 1) & (df_fits['action_time'].notna()),'after_fit'] = 1
# ['after_fit']

In [ ]:
df_fits.loc[df_fits['after_fit'] == 1, 'action_time'] = 0

# Вывод по конкретному аппликанту
df_fits[df_fits['id_applicant']== 4886560]

In [ ]:
# Статистика по аппликанту
applicants_g =  df_fits.dropna(subset=['action_time']).groupby(['id_applicant'])['action_time'].sum()
applicants_g.describe()

# applicants_g =  df_fits.dropna(subset=['action_time']).groupby(['id_applicant'])['action_time', 'is_fits'].sum()
# applicants_g[applicants_g['is_fits']>0].describe()

# applicants_g[applicants_g['is_fits']==0].describe()

In [ ]:
# df['action_time'][df['action_time'] < 0] = None
# Статистика по стадиям
stage_threshold = 0
stages_g =  df_fits.dropna(subset=['action_time']).groupby(['stage_name'])['action_time'].describe() # ['stage_name', 'activity_name']
stages_g[stages_g['count'] > stage_threshold]

In [ ]:
# Статистика по активностям
activity_threshold = 0
activities_g =  df_fits.dropna(subset=['action_time']).groupby(['activity_name'])['action_time'].describe()
activities_g[activities_g['count'] > activity_threshold]

In [ ]:
# Статисика по результатам
result_threshold = 0
results_g =  df_fits.dropna(subset=['action_time']).groupby(['result_name'])['action_time'].describe()
results_g[results_g['mean'] > 1]

# Обнуляем время в результатах без предсказуемой последующей обработки

In [ ]:
# Недозвон? Нет обратной связи? Ожидание?
results_empty = ['Резерв', 'Не подходит', 'Не принят', 'Отказ кандидата', 'Отказ руководителя', 'Принят', 'Кадровый резерв']
for res in results_empty:
    df_fits.loc[(df_fits['result_name'] == res) & df_fits['action_time'].notna(), 'action_time'] = 0
    
df_fits.loc[(df_fits['stage_name'] == 'Создание кандидата') & df_fits['action_time'].notna(), 'action_time'] = 0


#TTH
df_tth =  df_fits.dropna(subset=['action_time']).groupby(['id_applicant', 'id_partner', 'department_id'])['action_time'].sum().reset_index()
df_tth.rename(columns={"action_time": "TTH"}, inplace = True)
# df_tth['TTH'].to_excel('TTH.xlsx')
# df_tth = pd.merge(df_tth, df_fits[['id_applicant', 'id_partner']], how = 'right', on="id_applicant")
# df_tth = df_tth.join(df_fits.set_index('id_applicant')['id_partner'], on='id_applicant')
df_tth

In [ ]:
# TTH по департаментам
df_tth_dep =  df_fits.dropna(subset=['action_time']).groupby(['id_applicant', 'department_id'])['action_time'].count().reset_index()
# df_tth_dep = df_tth_dep.join(df_tth.set_index('id_applicant'), on='id_applicant')
# df_tth_dep = pd.merge(df_tth_dep, df_tth[['id_applicant','action_time']], how = 'left', on="id_applicant")
df_tth_dep = df_tth_dep.groupby(['department_id']).agg({'id_applicant':'count', 'action_time':'median'}).reset_index()
# df_tth_dep.to_excel('TTH_DEP.xlsx')
df_tth_dep.rename(columns={"id_applicant": "applicants","action_time": "TTH_median"}, inplace = True)
df_tth_dep

In [ ]:
df_fits['place'] = range(0, len(df_fits))
df_places = df_fits.groupby(['id_applicant'])['place'].min().reset_index()
df_places = df_places.rename(columns={"place": 'place_min'})
df_fits = pd.merge(df_fits, df_places, how = 'left', on="id_applicant")
df_fits['place'] = df_fits['place'] - df_fits['place_min']
df_fits = df_fits.drop(['place_min'] , axis=1)



from sklearn.ensemble import IsolationForest

print(len(df_tth['id_partner'].unique()))
print(len(df_tth['id_applicant'].unique()))
forest_model=IsolationForest(random_state=0, contamination=.01).fit(df_tth[['TTH']])
df_tth['anomaly'] = forest_model.predict(df_tth[['TTH']])
df_tth
# df_fits

In [ ]:
# График TTH IsolationForest без аномалий
def SetTTHLabels(table_name = 'TTH'):
    ax.set_title(table_name)
    ax.set_xlabel('TTH (Дни)')
    ax.set_ylabel('Количество')
    
fig, ax = plt.subplots(1, 1,figsize=(15,10))
bins = 50
ax.hist(df_tth['TTH'][(df_tth['anomaly']==1)&(df_tth['TTH']<1000)],bins=bins, color = "#00b5dd")
SetTTHLabels('TTH IsolationForest без аномалий')
plt.show()

In [ ]:
# График TTH IsolationForest с аномалиями
fig, ax = plt.subplots(1, 1,figsize=(15,10))
bins = 50
ax.hist(df_tth['TTH'][(df_tth['anomaly']==-1)&(df_tth['TTH']<1000)],bins=bins, color = "#00b5dd")
SetTTHLabels('TTH IsolationForest с аномалиями')
plt.show()

In [ ]:
# Запускать один раз!
treshold_down = 0.02
treshold_up = df_tth['TTH'].quantile(0.99)
print("Верхняя граница = "+ str(treshold_up))
df_fits = pd.merge(df_fits, df_tth[['id_applicant', 'TTH']], how = 'left', on="id_applicant")
df_tth = pd.DataFrame(df_tth).drop(columns = 'anomaly')

df_tth_norm = df_tth[(df_tth['TTH']<treshold_up)&(df_tth['TTH']>treshold_down)]
df_tth_small = df_tth[(df_tth['TTH']<treshold_down)]

df_norm = df_fits[(df_fits['TTH']<treshold_up)&(df_fits['TTH']>treshold_down)]
df_small = df_fits[(df_fits['TTH']<treshold_down)]

In [ ]:
print("Количество норм аппликантов = "+ str(len(df_norm.id_applicant.unique())))
print("Количество коротких аппликантов = "+ str(len(df_small.id_applicant.unique())))

In [ ]:
# Отсортированное среднее по результатам
df_norm.groupby(['result_name'])['place'].mean().sort_values()

In [ ]:
# Отсортированное среднее по стадиям
df_norm.groupby(['stage_name'])['place'].mean().sort_values()

In [ ]:
# Отсортированное среднее по активностям
df_norm.groupby(['activity_name'])['place'].mean().sort_values()

In [ ]:
# Какие-то графики
def vizScatTTH(df, xColumns, yColumn,  x_c, y_c):
    fig, axs = plt.subplots( y_c, x_c, figsize=(15,15))
    df = df[df[yColumn] <82]
    i = 0
    for column in xColumns:
        x = int(i / x_c)
        y = i % x_c    
        axs[x,y].set_ylabel("TTH")
        axs[x,y].set_xlabel(column)
        axs[x,y].scatter(df[column],df[yColumn],s=3, label="original")
        i += 1
    plt.show()

def columnDiscraber(df, column_name, x_col, y_col):
    activites =  df.groupby([column_name])['place'].mean().sort_values().reset_index()[column_name].values

    df_activity_mean = df.groupby([column_name, 'id_applicant'])['action_time'].mean().sort_values().reset_index()
    df_activity_mean = df_activity_mean.loc[df_activity_mean[column_name].isin(activites)]

    temp_df = pd.get_dummies(df_activity_mean[column_name]).rename(columns=lambda x: str(x))
    df_activity_mean =  pd.concat((df_activity_mean, temp_df), axis=1)

    for activity in activites:
        df_activity_mean.loc[df_activity_mean[activity]==1, activity] = df_activity_mean['action_time'] 
    df_activity_mean.fillna(0)
    df_activity_mean = df_activity_mean.groupby(['id_applicant']).sum().reset_index()
    df_activity_mean = df_activity_mean.rename(columns={"action_time": 'TTH'})
    vizScatTTH(df_activity_mean, activites,'TTH', x_col, y_col)

In [ ]:
# По активностям
columnDiscraber(df_norm, 'activity_name', 5, 4)

In [ ]:
# По стадиям
columnDiscraber(df_norm, 'stage_name', 4, 4)

In [ ]:
# Похоже что нормирование

# df_norm[df_norm['activity_name'] == 'Назначить время на оформление/медосмотр']
# df_test_g = df_norm[df_norm['activity_name'] == 'Назначить время на оформление/медосмотр'].groupby(['id_applicant','activity_name']).index.count().reset_index()
# df_test_g[df_test_g['index']>2]

In [ ]:
def dfAllDays(df):
    date_min = df['action_date'].min().tzinfo.localize(pd.to_datetime(20180101, format='%Y%m%d'))
    day_max = int(toDays(df['action_date'].max() - date_min)) + 1 # In days
    range_days = range(0, day_max)
    return pd.DataFrame(range_days, range_days, columns =['days']) 
    
    
    
def stageInDays(stage_action_name, df, show_scatter = True, up_date_treshold = 730, applicants = False):
    stage_date = df[df['stage_name'] == stage_action_name].groupby(['id_applicant'])['action_date'].max().reset_index()
    stage_date = df.loc[df['action_date'].isin(stage_date['action_date']), ['id_applicant','action_date','action_time']]
    stage_date = stage_date.rename(columns={"action_date": 'max_date'})
    stage_date = pd.merge(stage_date,df[df['stage_name'] == stage_action_name].groupby(['id_applicant'])['action_date'].min().reset_index(), how = 'left', on="id_applicant")
    stage_date = stage_date.rename(columns={"action_date": 'min_date'})

    date = stage_date['min_date'].min().tzinfo.localize(pd.to_datetime(20180101, format='%Y%m%d'))
#     date_20 = stage_date['min_date'].min().tzinfo.localize(pd.to_datetime(20200101, format='%Y%m%d'))
#     print(toDays(date_20 - date))# - toDays(stage_date['min_date'].min() - date))
    
    stage_date['stage_time_min'] = (toDays(stage_date['min_date'] - date)).astype(int) # In days
    stage_date['stage_time_max'] = (toDays(stage_date['max_date'] - date)).astype(int) # In days
#     stage_date.sort_values(by=['stage_time_min'], ascending=[False])

    applicant_days = dfAllDays(df)
    applicant_days['applicants'] = 0
    for cur_date in applicant_days['days']:
        applicant_days.loc[cur_date,'applicants'] = len(stage_date[(stage_date['stage_time_max']>= cur_date)&(stage_date['stage_time_min']<= cur_date)])
        
    applicant_days = applicant_days[applicant_days['days'] < up_date_treshold]
    
    stage_date = pd.merge(stage_date, applicant_days.rename(columns={"days": 'stage_time_min'}), how = 'left', on="stage_time_min")
    if show_scatter:
        plt.subplots(figsize=(15,10))
        plt.scatter(applicant_days['days'], applicant_days['applicants'], linewidth=1, label="original",s=5)
        plt.title("Стадия "+ stage_action_name)
        plt.xlabel('Дни с начала 2018 года')
        plt.ylabel('Количество аппликантов на стадии')
        plt.grid(True)
        plt.show()
    if applicants:
        return applicant_days
    else:
        return stage_date

In [ ]:
# График качества длинны обратотки аппликантов

long_stages = ['Оформление', 'СБ', 'Назначение времени на оформление/медосмотр', 'Интервью с руководителем']
# for stage in long_stages:
#     stageInDays(stage, df_norm, True, 9999)
df_long_stages = dfAllDays(df_norm)

plt.subplots(figsize=(15,10))
for stage in long_stages:
    df_long_stage = stageInDays(stage, df_norm, False, 9999, True)
    df_long_stages = pd.merge(df_long_stages, df_long_stage, how = 'left', on="days") 
    new_col_name = stage+ ' аппликатнов'
    df_long_stages =  df_long_stages.rename(columns={"applicants": new_col_name})
    plt.plot(df_long_stages['days'], df_long_stages[new_col_name], label="Стадия " + stage, linewidth=0.5)
#     ,s=1, linewidth=1
    
plt.title("Аппликанты по наидольшим стадиям")
plt.xlabel('Дни с начала 2018 года')
plt.ylabel('Количество аппликантов на стадии')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show() 
# df_long_stages.to_excel('LONG_STAGES.xlsx')
# df_long_stages

In [ ]:
# Таблица по графику отсортированные по дням
df_long_stages.sort_values(by =['days'])

In [ ]:
# Такой же график, но по среднему

mask = np.isin(df_long_stages.columns, ['days'], invert = True)
df_long_stages_col = df_long_stages.columns[mask]
df_long_stages[df_long_stages_col] = df_long_stages[df_long_stages_col].ewm(com=3.5).mean()

plt.subplots(figsize=(15,10))
for stage in long_stages:
    new_col_name = stage + ' аппликатнов'
    df_long_stages =  df_long_stages.rename(columns={"applicants": new_col_name})
    plt.plot(df_long_stages['days'], df_long_stages[new_col_name], label="Стадия " + stage, linewidth=0.5)
#     ,s=1, linewidth=1
    
plt.title("Аппликанты по наидольшим стадиям (Среднее по неделе)")
plt.xlabel('Дни с начала 2018 года')
plt.ylabel('Количество аппликантов на стадии')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show() 

# df_long_stages.to_excel('LONG_STAGES_MEAN(week).xlsx')

In [ ]:
# Такая же таблица по графику
df_long_stages.sort_values(by =['days'])

In [ ]:
date_min = df_norm['action_date'].min().tzinfo.localize(pd.to_datetime(20180101, format='%Y%m%d'))
df_norm['days'] = (toDays(df_norm['action_date'] - date_min)).astype(int)  
df_norm = pd.merge(df_norm, df_long_stages, how = 'left', on="days")
# df_fits.sort_values(by =['days'])


def addTimeToDF(df, column_name, result_name, new_res_name, temp):
    temp = df[df[column_name]==result_name]
    temp = temp.dropna(subset=['action_time']).groupby(['id_applicant'])['action_time'].sum().reset_index()
    temp = temp.rename(columns={"action_time": new_res_name})
    df = pd.merge(df, temp, how = 'left', on="id_applicant")
    df.fillna(value = {new_res_name:0}, inplace = True)
    return df
def dfMeanTime(df, column_name, result, result_new_name):
    df_mean = df[df[column_name]==result].groupby(['id_applicant'])['action_time'].mean().reset_index()
    df_mean = df_mean.rename(columns={"action_time": result_new_name})
    return df_mean
def dfRepitable(df, column_name, result_name, new_result_name):
    temp_df = df_norm[df_norm[column_name]==result_name].groupby(['id_applicant', 'activity_name', 'stage_name'])['action_date'].count().reset_index()
    temp_df = temp_df.rename(columns={"action_date": "amount"})
    temp_df = temp_df[temp_df['amount']>1]
    df = pd.merge(df, temp_df.groupby(['id_applicant'])['amount'].sum(), how = 'left', on="id_applicant")
    df = df.rename(columns={"amount": "amount_over_2calls"})
    df = pd.merge(df, temp_df.groupby(['id_applicant'])['amount'].count(), how = 'left', on="id_applicant")
    df = df.rename(columns={"amount": "count_over_2calls"})
    df['mean_over_2calls'] = df['amount_over_2calls'] / df['count_over_2calls'] - 1
    df[df['mean_over_2calls'] == -1] = 0
    df = df.drop(['amount_over_2calls', 'count_over_2calls'] , axis=1)
    df = df.rename(columns={"mean_over_2calls": new_result_name})
    return df


# Нормализация
fits_min =  df_norm[df_norm['result_name'] == 'Принят'].groupby(['id_applicant'])['action_date'].min().reset_index()




# Был ли чел в резерве
temp_df = df[df['stage_name']=='Резерв'].id_partner.unique()
df_reg = df_tth_norm.reset_index(drop=True)
# df_reg['Reserv'] = 0
# df_reg.loc[df_reg['id_partner'].isin(temp_df), 'Reserv']= 1

# Сезонность? (В какой месяц?) - 11 бинарных переменных
df_reg = pd.merge(df_reg, fits_min[['id_applicant','action_date']], how = 'left', on="id_applicant")
df_reg['month_start'] = df_reg['action_date'].dt.month
# df_reg['year_start'] = df_reg['action_date'].dt.year - 2018
df_reg = df_reg.drop(['action_date'] , axis=1)

temp_df = pd.get_dummies(df_reg["month_start"]).rename(columns=lambda x: 'month '+str(int(x)))
# temp_df = pd.concat((pd.get_dummies(df_reg["year_start"]).rename(columns=lambda x: 'year '+str(int(x))),temp_df), axis=1)
df_reg =  pd.concat((temp_df, df_reg), axis=1)
df_reg = df_reg.drop(['month_start'] , axis=1)
# df_reg = df_reg.drop(['year_start'] , axis=1)

# Департаменты? (количество нанятых в департаменте)
# df_reg = pd.merge(df_reg, df_tth_dep[['department_id','applicants']], how = 'left', on="department_id")
# df_reg = df_reg.rename(columns={"applicants": "dep_applicants"})

# Узнать результаты интервью с руководителем (Время) / Нет обратной связи (От руководителя) - ожидание
#  Узнать результаты интервью с руководителем
# df_reg = addTimeToDF(df_reg, 'activity_name','Узнать результаты интервью с руководителем', "interview_result_time", temp_df)
#  Нет обратной связи от руководителя
# df_reg = addTimeToDF(df_reg, 'result_name','Нет обратной связи от руководителя', "no_interview_info_time", temp_df)
#  Узнать результаты интервью с руководителем
# df_reg = addTimeToDF(df_reg, 'stage_name','Оформление', "oform_time", temp_df)

# Источник Яндекс цифра
df_reg['yandex_source'] = 0
# yandex_source id_applicants
temp_df = df.loc[df['result_name']=='Источник Яндекс Цифра', 'id_applicant'].unique() 
df_reg.loc[df_reg['id_applicant'].isin(temp_df), 'yandex_source'] = 1

# Сколько раз повторяются этапы на стадии (activity + stage name)
df_reg = dfRepitable(df_reg,'stage_name','Интервью с руководителем', 'Интервью недозвоны' )
df_reg = dfRepitable(df_reg,'stage_name','Оформление', 'Оформление недозвоны' )
df_reg = dfRepitable(df_reg,'stage_name','СБ', 'СБ недозвоны' )
df_reg = dfRepitable(df_reg,'stage_name','Назначение времени на оформление/медосмотр', 'Медосмотр недозвоны' )
df_reg = dfRepitable(df_reg,'stage_name','Обработка', 'Обработка недозвоны' )
# temp_df = df_norm[df_norm['stage_name']=='Интервью с руководителем'].groupby(['id_applicant', 'activity_name', 'stage_name'])['action_date'].count().reset_index()
# temp_df = temp_df.rename(columns={"action_date": "amount"})
# temp_df = temp_df[temp_df['amount']>1]
# df_reg = pd.merge(df_reg, temp_df.groupby(['id_applicant'])['amount'].sum(), how = 'left', on="id_applicant")
# df_reg = df_reg.rename(columns={"amount": "amount_over_2calls"})
# df_reg = pd.merge(df_reg, temp_df.groupby(['id_applicant'])['amount'].count(), how = 'left', on="id_applicant")
# df_reg = df_reg.rename(columns={"amount": "count_over_2calls"})
# df_reg['mean_over_2calls'] = df_reg['amount_over_2calls'] / df_reg['count_over_2calls'] - 1
# df_reg[df_reg['mean_over_2calls'] == -1] = 0
# df_reg = df_reg.drop(['amount_over_2calls', 'count_over_2calls'] , axis=1)


# Количество стадий
# temp_df = df_norm.groupby(['id_applicant', 'stage_name'])['action_date'].count().reset_index()
# temp_df = temp_df.groupby(['id_applicant']).agg({'action_date': ['sum','count']}).reset_index()
# temp_df.columns = temp_df.columns.droplevel(1)
# temp_df.columns.values[1] = "steps_amount"
# temp_df.columns.values[2] = "stages_amount"
# df_reg = pd.merge(df_reg, temp_df, how = 'left', on="id_applicant")

# Количество аппликантов в среднем за неделю
df_reg = pd.merge(df_reg, df_norm.groupby(['id_applicant'])[df_long_stages_col].mean().reset_index(), how = 'left', on="id_applicant")
df_reg = pd.merge(df_reg, dfMeanTime(df_norm, 'stage_name','СБ', "СБ (время)" ), how = 'left', on="id_applicant")
df_reg = pd.merge(df_reg, dfMeanTime(df_norm, 'stage_name','Интервью с руководителем', "Интервью с руководителем (время)" ), how = 'left', on="id_applicant")
# df_reg = pd.merge(df_reg, dfMeanTime(df_norm, 'stage_name','Оформление', "Оформление (время)" ), how = 'left', on="id_applicant")

# df_reg['Оформление'] = df_reg['Оформление аппликатнов']*df_reg['Оформление (время)']
# df_reg['СБ (на стадии/всего)'] = df_reg['СБ аппликатнов'] / len(df_reg[df_reg['СБ (время)']>0])
df_reg['СБ (время/среднее)'] = df_reg['СБ (время)'] / df_reg['СБ (время)'].mean()
# df_reg['СБ метрика'] = df_reg['СБ (на стадии/всего)'] * df_reg['СБ (время/среднее)']

df_reg['Интервью с руководителем аппликатнов (на стадии/всего)'] = df_reg['Интервью с руководителем аппликатнов'] / len(df_reg[df_reg['Интервью с руководителем (время)']>0])
df_reg['Интервью с руководителем (время/среднее)'] = df_reg['Интервью с руководителем (время)'] / df_reg['Интервью с руководителем (время)'].mean()
# df_reg['Интервью с руководителем метрика'] = df_reg['Интервью с руководителем аппликатнов (на стадии/всего)'] * df_reg['Интервью с руководителем (время/среднее)'] 

# df_reg['Интервью с руководителем'] = df_reg['Интервью с руководителем аппликатнов']*df_reg['Интервью с руководителем (время)']
# df_reg = df_reg.drop(['Оформление аппликатнов', 'Оформление (время)'] , axis=1)
df_reg = df_reg.drop(['Интервью с руководителем (время)'] , axis=1)
df_reg = df_reg.drop(['СБ (время)'] , axis=1)
# df_reg = df_reg.drop(['СБ (на стадии/всего)', 'СБ (время/среднее)'] , axis=1)

# df_reg = df_reg.drop(['Интервью с руководителем аппликатнов (на стадии/всего)', 'Интервью с руководителем (время/среднее)'] , axis=1)
df_reg = df_reg.drop(['Интервью с руководителем аппликатнов (на стадии/всего)'] , axis=1)
df_reg = df_reg.drop(['Интервью с руководителем аппликатнов', 'СБ аппликатнов', 'Назначение времени на оформление/медосмотр аппликатнов', 'Оформление аппликатнов'] , axis=1)

# Время по стадиям
# stages = ['Обработка', 'Интервью с руководителем']
# for stage in stages:
# # for stage in long_stages:
#     mean_stage = df_norm[df_norm['stage_name']==stage].dropna(subset=['action_time'])['action_time'].mean()
# #     print(mean_stage)
#     df_stage_mean = df_norm[df_norm['stage_name']==stage].dropna(subset=['action_time']).groupby(['id_applicant'])['action_time'].mean().reset_index()
#     df_stage_mean['action_time'] = df_stage_mean['action_time'] / mean_stage
#     df_stage_mean = df_stage_mean.rename(columns = {'action_time':stage + ' % от среднего'})
#     df_reg = pd.merge(df_reg, df_stage_mean, how = 'left', on="id_applicant")
# stage = 'Провести первичную обработку'
# stage = 'Обработка'
# mean_stage = df_norm[df_norm['stage_name']==stage].dropna(subset=['action_time'])['action_time'].mean()
# df_stage_mean = df_norm[df_norm['stage_name']==stage].dropna(subset=['action_time']).groupby(['id_applicant'])['action_time'].mean().reset_index()
# df_stage_mean['action_time'] = df_stage_mean['action_time'] / mean_stage
# df_stage_mean = df_stage_mean.rename(columns = {'action_time':stage + ' % от среднего'})
# df_reg = pd.merge(df_reg, df_stage_mean, how = 'left', on="id_applicant")

# temp_df
print(df_reg.isna().sum())
df_reg = df_reg.fillna(0)
df_reg

In [ ]:
temp_df = df_norm.groupby(['id_applicant', 'activity_name', 'stage_name'])['action_date'].count().reset_index()
temp_df = temp_df.rename(columns={"action_date": "amount"})
temp_df = temp_df[temp_df['amount']>1]
temp_df.groupby('stage_name').sum()
# temp_df[temp_df['stage_name'] == 'Обработка'].groupby('activity_name').count()

In [ ]:
def lm(df_reg, yColumn, xColumns_del = [], scaler = preprocessing.StandardScaler()):
    
    xColumn = df_reg.columns
    yColumn = yColumn
    xColumns_del.append(yColumn)
    mask = np.isin(xColumn, xColumns_del, invert = True)
    xColumn = xColumn[mask]
    
    df_reg2 = df_reg.loc[:, df_reg.columns != yColumn]
    df_reg.loc[:, df_reg.columns != yColumn] = pd.DataFrame(scaler.fit_transform(df_reg2), columns=df_reg2.columns, index = df_reg2.index.values)
        
    lr = LinearRegression()

    X_c = sm.add_constant(df_reg[xColumn])

    model = sm.OLS(df_reg[yColumn],X_c)
    fitted_model = model.fit()
    
    return fitted_model, xColumn, fitted_model.predict(X_c)

def vizScatTTHPred(df, predicted, xColumns, yColumn):
    xy = math.ceil(math.sqrt(len(xColumns)))
    if xy <2:
        xy = 2
    fig, axs = plt.subplots(xy, xy, figsize=(13,13))
    i = 0
    for column in xColumns:
        x, y =divmod(i, xy)    
        axs[x,y].set_ylabel("TTH")
        axs[x,y].set_xlabel(column)
        axs[x,y].scatter(df[column], df[yColumn],s=3, label="original")
        axs[x,y].scatter(df[column], predicted,s=3, label="predicted")
        i += 1
    plt.show()
    
def vizResTTHPred(df, predicted, xColumns, yColumn):
    xy = math.ceil(math.sqrt(len(xColumns)))
    if xy <2:
        xy = 2
    fig, axs = plt.subplots(xy, xy, figsize=(13,13))
    i = 0
    df.loc[:, 'res'] = df[yColumn] - predicted
    for column in xColumns:
        x, y =divmod(i, xy)    
        axs[x,y].set_ylabel("TTH остатки")
        axs[x,y].set_xlabel(column)
        axs[x,y].scatter(df[column],df['res'],s=3)
        i += 1
    plt.show()

    
yColumn = "TTH"
fitted_model, xColumn, predicted = lm(df_reg,yColumn,['id_partner', 'department_id', 'id_applicant', 'TTH', 'year 2', 'month 12', 'yandex_source'])
fitted_model.summary()

In [ ]:
# xy = 3
not_column = []
for column in xColumn:
    if 'month' in column:
        not_column.append(column)
not_column.append('Reserv')
mask = np.isin(xColumn, not_column, invert = True)
viz_column = xColumn[mask]

vizScatTTHPred(df_reg, predicted, viz_column, yColumn)
vizResTTHPred(df_reg,predicted,viz_column,yColumn)

In [ ]:
# Названия графиков
df_reg[viz_column].columns

In [ ]:
classifiers = [
    linear_model.BayesianRidge(),
    linear_model.ARDRegression(),
    linear_model.PassiveAggressiveRegressor(),
    linear_model.TheilSenRegressor(),
    linear_model.LinearRegression()]

# for classifier in classifiers:
#     classifier.fit(X_train, Y_train)
#     Y_pred = classifier.predict(X_test)
#     print(metrics.r2_score(Y_test, Y_pred))


classifier = linear_model.ARDRegression()
reg_results = classifier.fit(X_train, Y_train)
reg_coef = pd.DataFrame(data=classifier.coef_, index = xColumn)
reg_coef.sort_values(by=[0], ascending=[False], inplace = True)
print(reg_coef)

In [ ]:
# x_ax = range(len(Y_test))
# X_test_c = sm.add_constant(X_test)
# Y_pred = fitted_model.predict(X_test_c)
# plt.subplots(figsize=(13,13))
# plt.scatter(x_ax, Y_test, linewidth=1, label="original",s=5)
# plt.scatter(x_ax, Y_pred, linewidth=1.1, label="predicted",s=5)
# plt.title("y-test and y-predicted data")
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')
# plt.legend(loc='best',fancybox=True, shadow=True)
# plt.grid(True)
# plt.show() 

In [ ]:
import seaborn as sns
plt.subplots(figsize=(10,10))
sns.heatmap(df_reg[np.append(xColumn,yColumn)].corr(),annot=True,linewidths = 1)
plt.show()

In [ ]:
df_tth

In [ ]:
# ХЗ что это
stages_g =  df_norm.dropna(subset=['action_time']).groupby(['stage_name'])['action_time'].describe() # ['stage_name', 'activity_name']
stages_g['sum'] = stages_g['mean'] * stages_g['count']
stages_g

In [ ]:
# Хз что это
date = df_norm['action_date'].min().tzinfo.localize(pd.to_datetime(20200101, format='%Y%m%d'))
df_norm[df_norm['action_date']>date].groupby('stage_name')['action_time'].sum()

In [ ]:
# Среднее время обработки аппликатов по времени
stage_pie_mask = stages_g['sum'] > 2200
stage_sum =stages_g[stage_pie_mask]['sum'].sum()
stage_sum

y = stages_g[stage_pie_mask]['sum'] / stage_sum
mylabels = stages_g[stage_pie_mask].index

plt.subplots(1, 1,figsize=(15,10))
plt.pie(y, labels = mylabels)
plt.show() 

In [ ]:
# df_stage_activ = df_fits.dropna(subset=['action_time']).groupby(['stage_name', 'activity_name']).describe().reset_index
# df_stage_activ[]

# df_stage_result = df_fits.dropna(subset=['action_time']).groupby(['stage_name', 'result_name']).count().reset_index()
# df_stage_result[df_stage_result['stage_name']=='Оформление']

In [ ]:
reserved_applicants = df.loc[df['result_name']=="Кадровый резерв", 'id_applicant'].unique()
reserved_applicants= np.append(reserved_applicants, df.loc[df['stage_name']=="Кадровый резерв", 'id_applicant'].unique())
reserved_applicants = np.append(reserved_applicants, df.loc[df['stage_name']=="Резерв", 'id_applicant'].unique())
reserved_applicants = np.unique(reserved_applicants)


sb_c = 1
int_c = 1
df_sb_int = df_fits[(df_fits['id_applicant'].isin(reserved_applicants))& ((df_fits['stage_name']=='СБ') | (df_fits['stage_name']=='Интервью с руководителем'))]
df_sb_int = df_sb_int.groupby(['id_applicant', 'stage_name'])['action_time'].sum().reset_index()
df_sb_int = pd.merge(df_sb_int[df_sb_int['stage_name'] == 'Интервью с руководителем'], df_sb_int[df_sb_int['stage_name'] == 'СБ'], how = 'outer', on="id_applicant")
df_sb_int['action_time_x(short)'] = df_sb_int['action_time_x']
df_sb_int.loc[(df_sb_int['action_time_x(short)']>sb_c), 'action_time_x(short)'] = sb_c
df_sb_int['action_time_y(short)'] = df_sb_int['action_time_y']
df_sb_int.loc[df_sb_int['action_time_y(short)']>int_c, 'action_time_y(short)'] = int_c
df_sb_int.rename(columns = {'action_time_x':'interview_time', 'action_time_x(short)':'interview_time(short)','action_time_y':'sb_time', 'action_time_y(short)':'sb_time(short)'}, inplace = True)
df_sb_int = df_sb_int.drop(['stage_name_x','stage_name_y'] , axis=1)
# df_sb_int.to_excel('reserved.xlsx')


# nedozvon = df_fits[['id_applicant','action_date', 'action_time']][(df_fits['result_name'] == 'Недозвон')]
# nedozvon['action_date'] = nedozvon['action_date'].dt.tz_localize(None)
# nedozvon.to_excel('Nedozvon.xlsx')
sum_time_med_c = 3
sum_time_medN_c = 3
sum_time_res_c = 3
df_med = df_fits[(df_fits['activity_name'] == 'Назначить время на оформление/медосмотр')].groupby(['activity_name', 'id_applicant']).agg({'action_time': ['sum','count']}).reset_index()
df_med.columns = df_med.columns.droplevel(1)
df_med = df_med.drop('activity_name', axis=1)
df_med.columns.values[1] = "sum_time_med"
df_med.columns.values[2] = "count_time_med"
print(len(df_med))
df_med2 = df_fits[(df_fits['activity_name'] == 'Назначить время на оформление/медосмотр (недозвон)')].groupby(['activity_name', 'id_applicant']).agg({'action_time': ['sum','count']}).reset_index()
df_med2.columns = df_med2.columns.droplevel(1)
df_med2 = df_med2.drop('activity_name', axis=1)
df_med2.columns.values[1] = "sum_time_medN"
df_med2.columns.values[2] = "count_time_medN"
print(len(df_med2))
df_med =  pd.merge(df_med, df_med2, how = 'outer', on="id_applicant")
df_med2 = df_fits.dropna(subset=['action_time'])[(df_fits['activity_name'] == 'Узнать результаты приема')].groupby(['activity_name', 'id_applicant']).agg({'action_time': ['sum','count']}).reset_index()
df_med2.columns = df_med2.columns.droplevel(1)
df_med2 = df_med2.drop('activity_name', axis=1)
df_med2.columns.values[1] = "sum_time_res"
df_med2.columns.values[2] = "count_time_res"
print(len(df_med2))
df_med =  pd.merge(df_med, df_med2, how = 'outer', on="id_applicant")
df_med['sum_time_med(short)'] = df_med['sum_time_med']
df_med.loc[(df_med['sum_time_med(short)']>sum_time_med_c), 'sum_time_med(short)'] = sum_time_med_c
df_med['sum_time_medN(short)'] = df_med['sum_time_medN']
df_med.loc[df_med['sum_time_medN(short)']>sum_time_medN_c, 'sum_time_medN(short)'] = sum_time_medN_c
df_med['sum_time_res(short)'] = df_med['sum_time_res']
df_med.loc[df_med['sum_time_res(short)']>sum_time_res_c, 'sum_time_res(short)'] = sum_time_res_c
# df_med.to_excel('med_time.xlsx')

In [ ]:
# По результатам
results_g =  df_fits.dropna(subset=['action_time']).groupby(['result_name'])['action_time'].describe()
results_g

In [ ]:
# По активностям
activities_g =  df_fits.dropna(subset=['action_time']).groupby(['activity_name'])['action_time'].describe()
activities_g

In [ ]:
# Статистика TTH по департаментам
df_new_tth = df_tth.reset_index(drop = True)
# df_med
df_new_tth = pd.merge(df_new_tth, df_med, how = 'left', on="id_applicant")
df_new_tth.fillna(0, inplace = True)
df_new_tth['TTH'] = df_new_tth['TTH'] - (df_new_tth['sum_time_med'] - df_new_tth['sum_time_med(short)'] + df_new_tth['sum_time_medN'] - df_new_tth['sum_time_medN(short)'] + df_new_tth['sum_time_res'] - df_new_tth['sum_time_res(short)'])
df_new_tth = df_new_tth[['id_applicant','TTH']] 

# df_sb_int
df_new_tth = pd.merge(df_new_tth, df_sb_int, how = 'left', on="id_applicant")
df_new_tth.fillna(0, inplace = True)
df_new_tth['TTH'] = df_new_tth['TTH'] - (df_new_tth['sb_time'] - df_new_tth['sb_time(short)'] + df_new_tth['interview_time'] - df_new_tth['interview_time(short)'])
df_new_tth = df_new_tth[['id_applicant','TTH']] 


# df_new_tth.to_excel('TTH_NEW.xlsx')
df_tth_new_dep =  df_fits.dropna(subset=['action_time']).groupby(['id_applicant', 'department_id'])['action_time'].count().reset_index()
df_tth_new_dep = df_tth_new_dep.join(df_new_tth.set_index('id_applicant'), on='id_applicant')
df_tth_new_dep = df_tth_new_dep.groupby(['department_id'])['TTH'].median().reset_index()
df_tth_new_dep.to_excel('TTH_DEP_NEW.xlsx')
df_tth_new_dep.describe()


In [ ]:
# Квантиль THH (без выбросов)
quantile_dep = df_tth_dep['TTH_median'].quantile(.999)
quantile_dep

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(15,10))
bins = 100
ax.hist(df_tth_dep['TTH_median'][(df_tth_dep['TTH_median']>quantile_dep)],bins=bins)
SetTTHLabels('Департаменты (0.1 процент департаментов)')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(15,10))
bins = 100
ax.hist(df_tth_dep['TTH_median'][(df_tth_dep['TTH_median']<=quantile_dep)],bins=bins)
SetTTHLabels('Департаменты (99.9 процентов департаментов)')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(15,10))
bins = 20
ax.hist(df_tth['TTH'][(df_tth['TTH']<=1)],bins=bins)
SetTTHLabels('Кандидаты, обработанные до одного дня')
plt.show()

In [ ]:
# df_tth[(df_tth['TTH']<0.0001)]
quantile_tth = df_tth['TTH'].quantile(.99)
quantile_tth

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(15,10))
ax.hist(df_tth['TTH'][(df_tth['TTH']<5000)&(df_tth['TTH']>quantile_tth)],bins=100)
SetTTHLabels('Кандидаты (0.1 процент кандидатов)')
plt.show()

In [ ]:
fig, ax = plt.subplots(1, 1,figsize=(15,10))
ax.hist(df_tth['TTH'][(df_tth['TTH']<quantile_tth)& (df_tth['TTH']>0)], bins = 100)
SetTTHLabels('Кандидаты (99.9 процент кандидатов)')
plt.show()

In [ ]:
df_fits['action_time'].quantile(.999)

In [ ]:
df_tth['TTH'][df_tth['TTH']>100]

# Комменты

In [ ]:
# stage_name = 'Интервью с руководителем'
# stage_series = df_fits['action_time'][df_fits['stage_name'] == stage_name]
# time_threshold = stages_g.loc[stage_name]['75%']

# df.iloc[stage_series[stage_series.values > time_threshold].index]


# amount_applicant =  df_fits.groupby(['id_partner','id_applicant']).size().reset_index()
# amount_applicant =  amount_applicant.groupby(['id_partner'])['id_applicant'].count().reset_index()
# amount_applicant[amount_applicant['id_applicant']>3]


# df.iloc[df[df['stage_name'] == 'Групповое собеседование и тестирование'].index+1]
# print(len(stages[stages['count'] > stage_threshold]))
# df.iloc[df[df['action_time'] < 0].index + 1]
# df[df['activity_name'] == 'Провести первичную обработку']

# Когорты

In [ ]:
df_min_date =df_fits.groupby(['id_applicant'])['action_date'].min().reset_index()
df_cohort = pd.merge(df_tth, df_min_date, how = 'left', on="id_applicant")
df_cohort.rename(columns = {'action_date':'creation_date'}, inplace = True)
fits_min =  df_fits[df_fits['result_name'] == 'Принят'].groupby(['id_applicant'])['action_date'].max().reset_index()
df_cohort = pd.merge(df_cohort, fits_min, how = 'left', on="id_applicant")
df_cohort.rename(columns = {'action_date':'applied_date'}, inplace = True)
# df_cohort['applied_date'] = df_cohort['applied_date'].dt.tz_localize(None)
# df_cohort['creation_date'] = df_cohort['creation_date'].dt.tz_localize(None)
date2018 = df_cohort['applied_date'].min().tzinfo.localize(pd.to_datetime(20180101, format='%Y%m%d'))
df_cohort['applied_date_days'] = (df_cohort['applied_date'] - date2018).dt.days % 365
df_cohort['creation_date_days'] = (df_cohort['creation_date'] - date2018).dt.days % 365

# df_cohort.to_excel('cohort_new.xlsx')

df_cohort

In [ ]:
# Непонятный код
date2018 = df_cohort['applied_date'].min().tzinfo.localize(pd.to_datetime(20180101, format='%Y%m%d'))
df_cohort['weeks'] = (df_cohort['applied_date'] - date2018).dt.days
df_cohort['weeks'] = (df_cohort['weeks'] % 365 / 7).astype(int)
df_cohort['TTH_int'] = (df_cohort['TTH']/7).astype(int)
# df_cohort[(df_cohort['TTH_int']>0)&(df_cohort['TTH_int']<10)]
# retention_pivot = pd.DataFrame(df_cohort.pivot_table(index='weeks',
#                                         columns='TTH_int',
#                                         values='TTH',
#                                         aggfunc='count'))
# [(df_cohort['TTH_int']>0)&(df_cohort['TTH_int']<10)]
# retention_pivot
# retention_pivot[1].plot()
# retention_pivot[1].index + 1



# retention_pivot = pd.DataFrame(df_cohort.pivot_table(index='weeks',
#                                         columns='TTH_int',
#                                         values='TTH',
#                                         aggfunc='count'))
# [(df_cohort['TTH_int']>0)&(df_cohort['TTH_int']<10)]
# retention_pivot.to_excel('cohort.xlsx')
# retention_pivot[1].plot()
# retention_pivot[1].index + 1

# retention_pivot[1]



# df_cohort['TTH_int'].max()
# pd.DataFrame(20180101)

# date = pd.to_datetime(20180101, format='%Y%m%d')
# date = df_cohort['applied_date'].min().tzinfo.localize(pd.to_datetime(20180101, format='%Y%m%d'))
# date
# df_cohort['action_date'].min().tzinfo

In [ ]:
# Гистограмма
def showHist(data, grouped_data, column_name = 'stage_name', col_threshold = 0, is_special = True, xy = 4):
    
    fig, axs = plt.subplots(xy, xy, figsize=(20,15))
    i = 0
    not_special_const = 5
    
    for col in data[column_name].unique():
        series = data['action_time'][data[column_name] == col]
        time_threshold = not_special_const
        if is_special and col in grouped_data.index:
            time_threshold = grouped_data.loc[col]['75%']
        col_len = len(series)
    
        if col_len > col_threshold:
            x, y =divmod(i, xy)
            i+=1
#             axs[x,y].set_ylabel('Кол-во')
            axs[x,y].set_xlabel(f'{col}, {col_len}')
            series[series.values < time_threshold].hist(ax=axs[x,y],bins=100)
        print('.', end = '')
    plt.show()

def checkShowHist(data, grouped_data, column_name = 'stage_name', col_threshold = 0, is_special = True, xy = 4):
    
    not_special_const = 5
    if is_special and len(data[column_name].unique()) != len(grouped_data):
        print('Not all columns in grouped_data')
    
    i = 0
    for col in data[column_name].unique():
        series = data['action_time'][data[column_name] == col]
        col_len = len(series)
    
        if col_len > col_threshold:
            x, y =divmod(i, xy)
            if x > xy or y > xy:
                print(f'Too many columns in data for diagramm {xy}*{xy}')                
            i+=1

In [ ]:
# По стадиям
showHist(df_fits, stages_g, 'stage_name', stage_threshold)

In [ ]:
# По активностям
showHist(df_fits, activities_g, 'activity_name', activity_threshold, True)

In [ ]:
# По результатам
showHist(df_fits, results_g, 'result_name', result_threshold, True, 6)